Jamison Hunter

June 8, 2023

Coding Dojo

# PCA Exercise: Core Assignment

In [43]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

In [7]:
# load the dataset
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')
# view the shape of the dataset
mnist.data.shape

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


(70000, 784)

In [10]:
mnist.data.head()

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
mnist.target.head()

0    5
1    0
2    4
3    1
4    9
Name: class, dtype: category
Categories (10, object): ['0', '1', '2', '3', ..., '6', '7', '8', '9']

Prepare the data for modeling. Scale and apply PCA to your data, while retaining 95% of the variance. Be sure not to leak information.

In [12]:
# checking data for any oddities that need to be fixed
mnist.data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Columns: 784 entries, pixel1 to pixel784
dtypes: float64(784)
memory usage: 418.7 MB


In [13]:
mnist.data.isna().sum()

pixel1      0
pixel2      0
pixel3      0
pixel4      0
pixel5      0
           ..
pixel780    0
pixel781    0
pixel782    0
pixel783    0
pixel784    0
Length: 784, dtype: int64

In [14]:
mnist.data.duplicated().sum()

0

In [15]:
X = mnist.data
y = mnist.target

In [20]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [21]:
# Instantiate Standard Scaler
scaler = StandardScaler()
# Fit & transform data.
scaled_df = scaler.fit_transform(X)

In [26]:
# Instantiate & fit data using PCA
pca = PCA(n_components=.95)
pca_processor = pca.fit(scaled_df)

Create Two KNN Models 

a. One that uses the PCA transformed data to predict which number each image shows.

b. One that uses the original data, without the PCA transformation (but, remember you still need to scale the data!)

In [29]:
knn = KNeighborsClassifier(n_neighbors=1)
knn_pipe_pca = make_pipeline(pca_processor, knn)

knn_pipe_pca.fit(X_train, y_train)
knn_pipe_pca.predict(X_test)

knn_train_score_pca = knn_pipe_pca.score(X_train, y_train)
knn_test_score_pca = knn_pipe_pca.score(X_test, y_test)
print(f"Train score: {knn_train_score_pca}")
print(f"Test score: {knn_test_score_pca}")

Train score: 1.0
Test score: 0.9730857142857143


In [31]:
knn = KNeighborsClassifier(n_neighbors=1)
knn_pipe_no_pca = make_pipeline(scaler, knn)

knn_pipe_no_pca.fit(X_train, y_train)
knn_pipe_no_pca.predict(X_test)

knn_train_score_no_pca = knn_pipe_no_pca.score(X_train, y_train)
knn_test_score_no_pca = knn_pipe_no_pca.score(X_test, y_test)
print(f"Train score: {knn_train_score_no_pca}")
print(f"Test score: {knn_test_score_no_pca}")

Train score: 1.0
Test score: 0.9448571428571428


Evaluate and Compare The Models

Use separate cells to make predictions using each model. Include the cell magic command: `%%time` at the top of your cells when making predictions to see which model can create predictions faster, the one trained on PCA data or the one trained on non-PCA data. Evaluate both models using multiple appropriate metrics.

'%%time' will output under the cell a count of how long it takes the code in that cell to run.

In [32]:
%%time
preds_pca = knn_pipe_pca.predict(X_test)

CPU times: user 24.6 s, sys: 78.4 ms, total: 24.7 s
Wall time: 15.4 s


In [33]:
%%time
preds_no_pca = knn_pipe_no_pca.predict(X_test)

CPU times: user 1min 33s, sys: 164 ms, total: 1min 33s
Wall time: 55.1 s


In [53]:
print(classification_report(preds_pca, y_test))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      1731
           1       0.99      0.98      0.99      2014
           2       0.97      0.98      0.97      1737
           3       0.96      0.97      0.96      1792
           4       0.97      0.97      0.97      1595
           5       0.97      0.97      0.97      1605
           6       0.99      0.98      0.99      1772
           7       0.98      0.97      0.97      1894
           8       0.95      0.98      0.96      1601
           9       0.96      0.96      0.96      1759

    accuracy                           0.97     17500
   macro avg       0.97      0.97      0.97     17500
weighted avg       0.97      0.97      0.97     17500



In [54]:
print(classification_report(preds_no_pca, y_test))

              precision    recall  f1-score   support

           0       0.98      0.96      0.97      1739
           1       0.99      0.97      0.98      2025
           2       0.93      0.96      0.95      1710
           3       0.94      0.93      0.93      1822
           4       0.94      0.94      0.94      1579
           5       0.92      0.93      0.92      1578
           6       0.98      0.96      0.97      1788
           7       0.94      0.94      0.94      1884
           8       0.90      0.95      0.93      1573
           9       0.93      0.90      0.91      1802

    accuracy                           0.94     17500
   macro avg       0.94      0.94      0.94     17500
weighted avg       0.95      0.94      0.94     17500



Answer The Following Questions

a. Which model performed the best on the test set?

b. Which model was the fastest at making predictions?

The KNN model with PCA was not only a more accurate model for this data set but it was also able to make predictions faster than the model without PCA.